In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, LSTM

In [2]:
ev_data= pd.read_csv('C:\\Users\\Himani\\Desktop\\pp/data.csv')

In [3]:
ev_data.isnull().sum()

id                         0
clusterId                  0
Date                       0
connectionTime             0
Num. Rep.                  0
disconnectTime             0
Time Diff                  0
Session Length             0
donechargingTime           1
kWhDelivered               0
sessionId                  0
siteId                     0
spaceID                    0
stationID                  0
timezone                   0
userID                   298
Precip Type                0
temperature                0
Avg. Humidity              0
Avg Wind Speed(mph)        0
Wind Bearing(degrees)      0
Visibility(km)             0
Loud Cover                 0
Pressure(millibar)         0
Daily summary              0
dtype: int64

In [4]:
ev_data["donechargingTime"] = ev_data["donechargingTime"].fillna(ev_data["donechargingTime"].mode()[0])

In [5]:
NAs = pd.concat([ev_data.isnull().sum()], axis=1, keys=["EV"])
NAs[NAs.sum(axis=1) > 0]

,EV
userID,298


In [6]:
ev_data.pop("userID")

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
294   NaN
295   NaN
296   NaN
297   NaN
298   NaN
Name: userID, Length: 299, dtype: float64

In [7]:
ev_data.pop("Date")

0      Wed, 25 Apr 2018
1      Wed, 25 Apr 2018
2      Wed, 25 Apr 2018
3      Wed, 25 Apr 2018
4      Wed, 25 Apr 2018
             ...       
294    Tue, 01 May 2018
295    Tue, 01 May 2018
296    Tue, 01 May 2018
297    Tue, 01 May 2018
298    Tue, 01 May 2018
Name: Date, Length: 299, dtype: object

In [8]:
ev_data.pop("connectionTime")

0       11:08:04 
1        13:45:10
2       13:45:50 
3       14:37:06 
4       14:40:34 
          ...    
294     16:52:54 
295      16:58:54
296      17:09:09
297      17:11:32
298     17:28:59 
Name: connectionTime, Length: 299, dtype: object

In [9]:
ev_data.pop("disconnectTime")

0       13:20:10 GMT
1       00:56:16 GMT
2       23:04:45 GMT
3       23:55:34 GMT
4       23:03:12 GMT
           ...      
294     01:05:41 GMT
295     01:05:19 GMT
296     00:22:14 GMT
297     20:10:03 GMT
298     23:42:22 GMT
Name: disconnectTime, Length: 299, dtype: object

In [10]:
ev_data.pop("Time Diff")

0      02:12:06
1      11:11:06
2      10:19:45
3      09:18:28
4      08:22:38
         ...   
294    08:12:37
295    08:06:25
296    07:13:05
297    02:58:31
298    06:13:23
Name: Time Diff, Length: 299, dtype: object

In [11]:
ev_data.pop("donechargingTime")

0      Wed, 25 Apr 2018 13:21:10 GMT
1      Wed, 25 Apr 2018 16:44:15 GMT
2      Wed, 25 Apr 2018 14:51:44 GMT
3      Wed, 25 Apr 2018 16:05:22 GMT
4      Wed, 25 Apr 2018 17:40:30 GMT
                   ...              
294    Tue, 01 May 2018 19:11:06 GMT
295    Tue, 01 May 2018 23:54:46 GMT
296    Tue, 01 May 2018 18:46:19 GMT
297    Tue, 01 May 2018 20:09:58 GMT
298    Tue, 01 May 2018 23:42:19 GMT
Name: donechargingTime, Length: 299, dtype: object

In [12]:
ev_data.pop("timezone")

0      America/Los_Angeles
1      America/Los_Angeles
2      America/Los_Angeles
3      America/Los_Angeles
4      America/Los_Angeles
              ...         
294    America/Los_Angeles
295    America/Los_Angeles
296    America/Los_Angeles
297    America/Los_Angeles
298    America/Los_Angeles
Name: timezone, Length: 299, dtype: object

In [13]:
ev_data.pop("sessionId")

0      2_39_78_362_2018-04-25 11:08:04.400812
1       2_39_95_27_2018-04-25 13:45:09.617470
2      2_39_79_380_2018-04-25 13:45:49.962001
3      2_39_79_379_2018-04-25 14:37:06.460772
4      2_39_79_381_2018-04-25 14:40:33.638896
                        ...                  
294    2_39_92_442_2018-05-01 16:52:53.742929
295    2_39_129_17_2018-05-01 16:58:53.921677
296    2_39_124_22_2018-05-01 17:09:09.120419
297    2_39_123_23_2018-05-01 17:11:32.081386
298    2_39_138_29_2018-05-01 17:28:59.219125
Name: sessionId, Length: 299, dtype: object

In [14]:
ev_data.pop("Precip Type")

0      rain
1      rain
2      rain
3      rain
4      rain
       ... 
294    rain
295    rain
296    rain
297    rain
298    rain
Name: Precip Type, Length: 299, dtype: object

In [15]:
ev_data.pop("Daily summary")

0      Mostly cloudy .
1      Mostly cloudy .
2      Mostly cloudy .
3      Mostly cloudy .
4      Mostly cloudy .
            ...       
294     Mostly cloudy 
295     Mostly cloudy 
296     Mostly cloudy 
297     Mostly cloudy 
298     Mostly cloudy 
Name: Daily summary, Length: 299, dtype: object

In [16]:
ev_data.pop("Loud Cover")

0      0
1      0
2      0
3      0
4      0
      ..
294    0
295    0
296    0
297    0
298    0
Name: Loud Cover, Length: 299, dtype: int64

In [17]:
ev_data

,id,clusterId,Num. Rep.,Session Length,kWhDelivered,siteId,spaceID,stationID,temperature,Avg. Humidity,Avg Wind Speed(mph),Wind Bearing(degrees),Visibility(km),Pressure(millibar)
0,5bc90cb9f9af8b0d7fe77cd2,39,11.13,2.20,7.932,2,CA-496,2-39-78-362,24.372222,34.846153,13.5400,300,9.9015,1004.55
1,5bc90cb9f9af8b0d7fe77cd3,39,13.75,11.19,10.013,2,CA-319,2-39-95-27,24.372222,34.846153,13.5400,300,9.9015,1004.55
2,5bc90cb9f9af8b0d7fe77cd4,39,13.76,10.33,5.257,2,CA-489,2-39-79-380,24.372222,34.846153,13.5400,300,9.9015,1004.55
3,5bc90cb9f9af8b0d7fe77cd5,39,14.62,9.31,5.177,2,CA-327,2-39-79-379,24.372222,34.846153,13.5400,300,9.9015,1004.55
4,5bc90cb9f9af8b0d7fe77cd6,39,14.68,8.38,10.119,2,CA-490,2-39-79-381,24.372222,34.846153,13.5400,300,9.9015,1004.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,5bc9160af9af8b0dad3c06a8,39,16.88,8.21,7.799,2,CA-498,2-39-92-442,28.511111,38.763024,12.4883,339,16.1000,1009.55
295,5bc9160af9af8b0dad3c06a9,39,16.98,8.11,11.304,2,CA-307,2-39-129-17,28.511111,38.763024,12.4883,339,16.1000,1009.55
296,5bc9160af9af8b0dad3c06aa,39,17.15,7.22,1.943,2,CA-312,2-39-124-22,28.511111,38.763024,12.4883,339,16.1000,1009.55
297,5bc9160af9af8b0dad3c06ab,39,17.19,2.98,7.053,2,CA-313,2-39-123-23,28.511111,38.763024,12.4883,339,16.1000,1009.55


In [18]:
ev_data.pop("id")

0      5bc90cb9f9af8b0d7fe77cd2
1      5bc90cb9f9af8b0d7fe77cd3
2      5bc90cb9f9af8b0d7fe77cd4
3      5bc90cb9f9af8b0d7fe77cd5
4      5bc90cb9f9af8b0d7fe77cd6
                 ...           
294    5bc9160af9af8b0dad3c06a8
295    5bc9160af9af8b0dad3c06a9
296    5bc9160af9af8b0dad3c06aa
297    5bc9160af9af8b0dad3c06ab
298    5bc9160af9af8b0dad3c06ac
Name: id, Length: 299, dtype: object

In [19]:
ev_data.pop("spaceID")

0      CA-496
1      CA-319
2      CA-489
3      CA-327
4      CA-490
        ...  
294    CA-498
295    CA-307
296    CA-312
297    CA-313
298    CA-304
Name: spaceID, Length: 299, dtype: object

In [20]:
ev_data.pop("stationID")

0      2-39-78-362
1       2-39-95-27
2      2-39-79-380
3      2-39-79-379
4      2-39-79-381
          ...     
294    2-39-92-442
295    2-39-129-17
296    2-39-124-22
297    2-39-123-23
298    2-39-138-29
Name: stationID, Length: 299, dtype: object

In [21]:
ev_data.dtypes

clusterId                  int64
Num. Rep.                float64
Session Length           float64
kWhDelivered             float64
siteId                     int64
temperature              float64
Avg. Humidity            float64
Avg Wind Speed(mph)      float64
Wind Bearing(degrees)      int64
Visibility(km)           float64
Pressure(millibar)       float64
dtype: object

In [22]:
ev_data.shape

(299, 11)

In [25]:
X = ev_data.drop(columns='kWhDelivered')
Y = ev_data['kWhDelivered']

In [26]:
ev_data.shape

(299, 11)

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

In [31]:
# layers
model = Sequential()
model.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
model.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear'))

In [39]:
model.compile(loss='mean_absolute_error', optimizer='adam')


In [40]:
# Train the model on the training dataset
model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
3/3 [==============================] - 1s 120ms/step - loss: 5.2482 - val_loss: 4.8660
Epoch 2/20
3/3 [==============================] - 0s 23ms/step - loss: 4.8725 - val_loss: 4.8389
Epoch 3/20
3/3 [==============================] - 0s 26ms/step - loss: 4.9576 - val_loss: 4.8385
Epoch 4/20
3/3 [==============================] - 0s 25ms/step - loss: 5.1423 - val_loss: 4.8453
Epoch 5/20
3/3 [==============================] - 0s 25ms/step - loss: 5.0426 - val_loss: 4.8598
Epoch 6/20
3/3 [==============================] - 0s 27ms/step - loss: 4.9910 - val_loss: 4.8788
Epoch 7/20
3/3 [==============================] - 0s 25ms/step - loss: 5.2095 - val_loss: 4.8866
Epoch 8/20
3/3 [==============================] - 0s 29ms/step - loss: 5.1179 - val_loss: 4.8699
Epoch 9/20
3/3 [==============================] - 0s 30ms/step - loss: 5.0541 - val_loss: 4.8523
Epoch 10/20
3/3 [==============================] - 0s 32ms/step - loss: 5.0451 - val_loss: 4.8463
Epoch 11/20
3/3 [===========

In [41]:
# Use the trained model to make predictions on the test dataset
y_pred = model.predict(X_test)


2/2 [==============================] - 0s 3ms/step


In [42]:
print(y_pred)

[[6.3649592]
 [6.2020297]
 [6.202463 ]
 [6.5295835]
 [6.3666854]
 [6.559687 ]
 [6.2924056]
 [6.3833723]
 [6.6749396]
 [6.5760994]
 [6.555305 ]
 [6.680191 ]
 [6.533632 ]
 [6.692982 ]
 [6.5527453]
 [6.691375 ]
 [6.2927766]
 [6.528967 ]
 [6.547935 ]
 [6.1507483]
 [6.559565 ]
 [6.62797  ]
 [6.549912 ]
 [6.361677 ]
 [6.6051006]
 [6.5411863]
 [6.189654 ]
 [6.663627 ]
 [6.5734706]
 [6.352879 ]
 [6.428868 ]
 [6.552996 ]
 [6.2006025]
 [6.206191 ]
 [6.5543303]
 [6.66169  ]
 [6.423525 ]
 [6.5573316]
 [6.6172338]
 [6.21232  ]
 [6.571049 ]
 [6.2093043]
 [6.349451 ]
 [6.567454 ]
 [6.3617306]
 [6.5208898]
 [6.289248 ]
 [6.210115 ]
 [6.664225 ]
 [6.4090295]
 [6.6025233]
 [6.5499825]
 [6.525069 ]
 [6.522555 ]
 [6.5744524]
 [6.558246 ]
 [6.6069255]
 [6.6766615]
 [6.5975494]
 [6.290648 ]]


In [43]:
def smape(actual, predicted):
    """
    Calculate SMAPE (Symmetric Mean Absolute Percentage Error) for actual and predicted values.
    """
    actual = np.array(actual)
    predicted = np.array(predicted)
    return np.mean((np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted))) * 2) * 100

In [44]:
smape_score = smape(Y_test, y_pred)
print("SMAPE score:", smape_score)

SMAPE score: 68.32849313174762


In [45]:
# R squared error
error_score = r2_score(Y_test, y_pred)
print("R squared error : ", error_score)

R squared error :  -0.17095324110287358
